<a href="https://colab.research.google.com/github/chunyuan0221/3rd-ML100days/blob/master/Day_51_53_Midterm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Day051-53 Midterm Test - Enron Fraud Dataset 安隆公司詐欺案資料集
***
- 專案目的：安隆公司曾是一間能源公司，2001 年破產前是世界上最大的電力、天然氣及電信公司之一。擁有上千億資產的公司於 2002 年竟然在短短幾周內宣告破產，才揭露其財報在多年以來均是造假的醜聞。在本資料集中你將會扮演偵探的角色，透過高層經理人內部的 mail 來往的情報以及薪資、股票等財務特徵，訓練出一個機器學習模型來幫忙你找到可疑的詐欺犯罪者是誰! 我們已經先幫你找到幾位犯罪者 (Person-of-Interest, poi) 與清白的員工，請利用這些訓練資料來訓練屬於自己的詐欺犯機器學習模型吧!

- 由於資料放置於私人Google雲端空間，因此必須先讓colab的資料路徑更改為Google雲端空間
- 點選認證連結並複製、貼上認證碼即可連結至私人雲端硬碟空間

In [0]:
from google.colab import drive

In [2]:
drive.mount('content')

Drive already mounted at content; to attempt to forcibly remount, call drive.mount("content", force_remount=True).


- 透過os.listdir可以查看路徑下的資料
- 找到存放訓練與預測資料的位置和路徑後，即可載入數據

In [0]:
import os

In [4]:
os.listdir('content/My Drive/ML100_Colab Notebooks/6.ML100_Midterm test')

['test_features.csv',
 'train_data.csv',
 'Day_51-53_Midterm.ipynb',
 'result.csv']

In [0]:
import pandas as pd
import numpy as np

In [6]:
# 設定為資料的根目錄
data_root = 'content/My Drive/ML100_Colab Notebooks/6.ML100_Midterm test/'
# load訓練資料 & 預測資料
train_data = pd.read_csv(data_root + 'train_data.csv')
test_features = pd.read_csv(data_root + 'test_features.csv')

print(f'train data shape:{train_data.shape}')
print(f'test features data shape:{test_features.shape}')

train data shape:(113, 22)
test features data shape:(33, 21)


### 資料前處理 - 查看資料內容
***
- 由上面結果可知:
  1. train data:有113筆資料，22個參數。
  2. test features:有33筆，21個參數。
  3. test相比train data的參數少了1個，那可能就是我們要預測的label。
- 我們可以進一步查看參數內容：少的參數是poi(犯罪機率)也就是我們要預測的label。


In [7]:
print(train_data.columns)
print(test_features.columns)

Index(['name', 'bonus', 'deferral_payments', 'deferred_income',
       'director_fees', 'email_address', 'exercised_stock_options', 'expenses',
       'from_messages', 'from_poi_to_this_person', 'from_this_person_to_poi',
       'loan_advances', 'long_term_incentive', 'other', 'poi',
       'restricted_stock', 'restricted_stock_deferred', 'salary',
       'shared_receipt_with_poi', 'to_messages', 'total_payments',
       'total_stock_value'],
      dtype='object')
Index(['name', 'bonus', 'deferral_payments', 'deferred_income',
       'director_fees', 'email_address', 'exercised_stock_options', 'expenses',
       'from_messages', 'from_poi_to_this_person', 'from_this_person_to_poi',
       'loan_advances', 'long_term_incentive', 'other', 'restricted_stock',
       'restricted_stock_deferred', 'salary', 'shared_receipt_with_poi',
       'to_messages', 'total_payments', 'total_stock_value'],
      dtype='object')


### 資料前處理 - 保留與清除資料欄位
***
1. 找到label欄位並另存為 y_train，再從原始欄位中清除。
2. 清除沒意義的欄位：獨立名字、索引ID等等(name、email_address)，但若是有可做為分組的名稱則有意義。
3. 保留數值欄位：不同的薪水、獎金、工作經驗等等的。


In [8]:
train_data.head()

,name,bonus,deferral_payments,deferred_income,director_fees,email_address,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,from_this_person_to_poi,loan_advances,long_term_incentive,other,poi,restricted_stock,restricted_stock_deferred,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value
0,RICE KENNETH D,1750000.0,NaN,-3504386.0,NaN,ken.rice@enron.com,19794175.0,46950.0,18.0,42.0,4.0,NaN,1617011.0,174839.0,True,2748364.0,NaN,420636.0,864.0,905.0,505050.0,22542539.0
1,SKILLING JEFFREY K,5600000.0,NaN,NaN,NaN,jeff.skilling@enron.com,19250000.0,29336.0,108.0,88.0,30.0,NaN,1920000.0,22122.0,True,6843672.0,NaN,1111258.0,2042.0,3627.0,8682716.0,26093672.0
2,SHELBY REX,200000.0,NaN,-4167.0,NaN,rex.shelby@enron.com,1624396.0,22884.0,39.0,13.0,14.0,NaN,NaN,1573324.0,True,869220.0,NaN,211844.0,91.0,225.0,2003885.0,2493616.0
3,KOPPER MICHAEL J,800000.0,NaN,NaN,NaN,michael.kopper@enron.com,NaN,118134.0,NaN,NaN,NaN,NaN,602671.0,907502.0,True,985032.0,NaN,224305.0,NaN,NaN,2652612.0,985032.0
4,CALGER CHRISTOPHER F,1250000.0,NaN,-262500.0,NaN,christopher.calger@enron.com,NaN,35818.0,144.0,199.0,25.0,NaN,375304.0,486.0,True,126027.0,NaN,240189.0,2188.0,2598.0,1639297.0,126027.0


In [9]:
train_data.dtypes

name                          object
bonus                        float64
deferral_payments            float64
deferred_income              float64
director_fees                float64
email_address                 object
exercised_stock_options      float64
expenses                     float64
from_messages                float64
from_poi_to_this_person      float64
from_this_person_to_poi      float64
loan_advances                float64
long_term_incentive          float64
other                        float64
poi                             bool
restricted_stock             float64
restricted_stock_deferred    float64
salary                       float64
shared_receipt_with_poi      float64
to_messages                  float64
total_payments               float64
total_stock_value            float64
dtype: object

In [10]:
# find 'poi'，建立y_trian
train_label = train_data[['poi']]  # 這樣才會是DataFrame格式
print(f'y_train shape:{train_label.shape}')

# 清除 name、email_address
train_data = train_data.drop(['name', 'email_address', 'poi'], axis=1)
print(f'x_train shape:{train_data.shape}')

train_data.head()

y_train shape:(113, 1)
x_train shape:(113, 19)


,bonus,deferral_payments,deferred_income,director_fees,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,from_this_person_to_poi,loan_advances,long_term_incentive,other,restricted_stock,restricted_stock_deferred,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value
0,1750000.0,NaN,-3504386.0,NaN,19794175.0,46950.0,18.0,42.0,4.0,NaN,1617011.0,174839.0,2748364.0,NaN,420636.0,864.0,905.0,505050.0,22542539.0
1,5600000.0,NaN,NaN,NaN,19250000.0,29336.0,108.0,88.0,30.0,NaN,1920000.0,22122.0,6843672.0,NaN,1111258.0,2042.0,3627.0,8682716.0,26093672.0
2,200000.0,NaN,-4167.0,NaN,1624396.0,22884.0,39.0,13.0,14.0,NaN,NaN,1573324.0,869220.0,NaN,211844.0,91.0,225.0,2003885.0,2493616.0
3,800000.0,NaN,NaN,NaN,NaN,118134.0,NaN,NaN,NaN,NaN,602671.0,907502.0,985032.0,NaN,224305.0,NaN,NaN,2652612.0,985032.0
4,1250000.0,NaN,-262500.0,NaN,NaN,35818.0,144.0,199.0,25.0,NaN,375304.0,486.0,126027.0,NaN,240189.0,2188.0,2598.0,1639297.0,126027.0


### 資料前處理 - 遺失值處理(NaN)
***
- 在觀察資料中，我們還發現有許多欄位都有遺失值(NaN)，處理遺失值有幾種方式：
  1. drop整欄：當欄位中遺失值太多時，此欄位的參考價值則不高。
  2. 填值：當只有部分資料遺失，可以填入平均值、中位數或眾數取代NaN。

In [11]:
train_data.describe()

,bonus,deferral_payments,deferred_income,director_fees,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,from_this_person_to_poi,loan_advances,long_term_incentive,other,restricted_stock,restricted_stock_deferred,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value
count,6.100000e+01,2.800000e+01,3.400000e+01,13.000000,8.100000e+01,73.000000,65.000000,65.000000,65.000000,2.000000e+00,4.900000e+01,6.900000e+01,8.200000e+01,10.000000,7.300000e+01,65.000000,65.000000,9.600000e+01,9.800000e+01
mean,1.147436e+06,6.344374e+05,-4.625664e+05,89397.846154,2.985081e+06,51040.547945,711.323077,64.800000,40.092308,4.096250e+07,7.926171e+05,4.471774e+05,1.294855e+06,-221885.700000,2.739025e+05,1111.369231,2156.061538,2.590977e+06,3.527136e+06
std,1.505189e+06,8.603646e+05,8.095392e+05,41143.391399,6.004174e+06,47596.682104,2074.497628,91.863214,88.901407,5.736404e+07,9.504645e+05,1.341564e+06,2.498335e+06,205191.374121,1.716647e+05,1165.852016,2811.676718,1.056645e+07,7.182997e+06
min,7.000000e+04,-1.025000e+05,-3.504386e+06,3285.000000,3.285000e+03,148.000000,12.000000,0.000000,0.000000,4.000000e+05,7.102300e+04,2.000000e+00,4.409300e+04,-560222.000000,4.770000e+02,2.000000,57.000000,1.480000e+02,-4.409300e+04
25%,4.500000e+05,7.656750e+04,-5.527032e+05,101250.000000,4.004780e+05,18834.000000,19.000000,10.000000,0.000000,2.068125e+07,2.750000e+05,9.720000e+02,2.689220e+05,-389621.750000,2.061210e+05,178.000000,517.000000,3.024025e+05,4.211518e+05
50%,7.500000e+05,1.951900e+05,-1.175340e+05,108579.000000,8.500100e+05,41953.000000,45.000000,28.000000,7.000000,4.096250e+07,4.221580e+05,5.238200e+04,4.628225e+05,-139856.500000,2.516540e+05,599.000000,1088.000000,1.106740e+06,9.979710e+05
75%,1.000000e+06,8.342052e+05,-2.708325e+04,112492.000000,2.165172e+06,59175.000000,215.000000,88.000000,27.000000,6.124375e+07,8.318090e+05,3.620960e+05,9.664905e+05,-77953.250000,2.885890e+05,1902.000000,2649.000000,1.985668e+06,2.493616e+06
max,8.000000e+06,2.964506e+06,-1.042000e+03,125034.000000,3.434838e+07,228763.000000,14368.000000,528.000000,411.000000,8.152500e+07,5.145434e+06,1.035973e+07,1.476169e+07,44093.000000,1.111258e+06,4527.000000,15149.000000,1.035598e+08,4.911008e+07


In [12]:
# 計數每個欄位中，有value的有多少個
train_data.count()

bonus                        61
deferral_payments            28
deferred_income              34
director_fees                13
exercised_stock_options      81
expenses                     73
from_messages                65
from_poi_to_this_person      65
from_this_person_to_poi      65
loan_advances                 2
long_term_incentive          49
other                        69
restricted_stock             82
restricted_stock_deferred    10
salary                       73
shared_receipt_with_poi      65
to_messages                  65
total_payments               96
total_stock_value            98
dtype: int64

- 可以發現在某些欄位具有很多遺失值，而真正的有值的不多。
- 這邊先都以中位數填值。

In [13]:
median_value = dict(train_data.median())
x_train = train_data.fillna(value=median_value)
x_train.head()

,bonus,deferral_payments,deferred_income,director_fees,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,from_this_person_to_poi,loan_advances,long_term_incentive,other,restricted_stock,restricted_stock_deferred,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value
0,1750000.0,195190.0,-3504386.0,108579.0,19794175.0,46950.0,18.0,42.0,4.0,40962500.0,1617011.0,174839.0,2748364.0,-139856.5,420636.0,864.0,905.0,505050.0,22542539.0
1,5600000.0,195190.0,-117534.0,108579.0,19250000.0,29336.0,108.0,88.0,30.0,40962500.0,1920000.0,22122.0,6843672.0,-139856.5,1111258.0,2042.0,3627.0,8682716.0,26093672.0
2,200000.0,195190.0,-4167.0,108579.0,1624396.0,22884.0,39.0,13.0,14.0,40962500.0,422158.0,1573324.0,869220.0,-139856.5,211844.0,91.0,225.0,2003885.0,2493616.0
3,800000.0,195190.0,-117534.0,108579.0,850010.0,118134.0,45.0,28.0,7.0,40962500.0,602671.0,907502.0,985032.0,-139856.5,224305.0,599.0,1088.0,2652612.0,985032.0
4,1250000.0,195190.0,-262500.0,108579.0,850010.0,35818.0,144.0,199.0,25.0,40962500.0,375304.0,486.0,126027.0,-139856.5,240189.0,2188.0,2598.0,1639297.0,126027.0


### 資料前處理
***
- 特徵工程：改善input特徵間的差距。
  - input特徵：所謂的特徵可以是每筆資料中的年齡、性別、學歷、薪水等等。當我們在預測貸款償還能力預測時，這些特徵可能就是影響因素。
  - 優點：
    1. 不同特徵的數值會有不同的大小(ex：薪水 -> 萬，年齡 -> 十位)，這樣的差距會影響模型訓練時間。
    2. 使不同特徵的數值都縮放到 -1~1 或 0-1 之間。
- Label Encoder：更改特徵名稱。
  - 特徵名稱：有的欄位特徵類型是object需要將類別轉換成int(進行編號：0、1、2...)。
- OHE(OneHotEncodeing)：更改特徵名稱。
  - 特徵名稱：類別經過Label Encoder轉換為編號後，會有距離問題。因此需要轉換：
        OHE：
        編號：1、2、3
        轉換後：001、010、100 (個為數代表1、十位數代表2、百位數代表3)

In [0]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score

In [0]:
MMEncoder = MinMaxScaler()
x_train_mm = MMEncoder.fit_transform(x_train)

SSEncoder = StandardScaler()
x_train_ss = SSEncoder.fit_transform(x_train)


In [16]:
LabEncoder = LabelEncoder()
y_train = LabEncoder.fit_transform(train_label)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:251: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


### 資料分割

In [17]:
# x不做特徵工程
Xtrain, Xtest, Ytrain, Ytest = train_test_split(x_train, y_train, test_size=0.3, random_state=42)
print(f'Xtrain shape:{Xtrain.shape}')
print(f'Xtest shape:{Xtest.shape}')
print(f'Ytrain shape:{Ytrain.shape}')
print(f'Ytest shape:{Ytest.shape}')

# MinMaxScaler
Xtrain_mm, Xtest_mm, Ytrain_mm, Ytest_mm = train_test_split(x_train_mm, y_train, test_size=0.3, random_state=42)
print(f'Xtrain shape:{Xtrain_mm.shape}')
print(f'Xtest shape:{Xtest_mm.shape}')
print(f'Ytrain shape:{Ytrain_mm.shape}')
print(f'Ytest shape:{Ytest_mm.shape}')

# StandarScalar
Xtrain_ss, Xtest_ss, Ytrain_ss, Ytest_ss = train_test_split(x_train_ss, y_train, test_size=0.3, random_state=42)
print(f'Xtrain shape:{Xtrain_ss.shape}')
print(f'Xtest shape:{Xtest_ss.shape}')
print(f'Ytrain shape:{Ytrain_ss.shape}')
print(f'Ytest shape:{Ytest_ss.shape}')

Xtrain shape:(79, 19)
Xtest shape:(34, 19)
Ytrain shape:(79,)
Ytest shape:(34,)
Xtrain shape:(79, 19)
Xtest shape:(34, 19)
Ytrain shape:(79,)
Ytest shape:(34,)
Xtrain shape:(79, 19)
Xtest shape:(34, 19)
Ytrain shape:(79,)
Ytest shape:(34,)


### 建立模型

In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
import warnings
warnings.filterwarnings('ignore')

In [0]:
logistic = LogisticRegression()
DTree = DecisionTreeClassifier()
RForest = RandomForestClassifier()
GDBT = GradientBoostingClassifier()

In [20]:
# 
print('x do not feature engineering')
print(f'LogisticRegression score:{cross_val_score(logistic, Xtrain, Ytrain, cv=5).mean()}')
print(f'DecisionTree score:{cross_val_score(DTree, Xtrain, Ytrain, cv=5).mean()}')
print(f'RandomForest score:{cross_val_score(RForest, Xtrain, Ytrain, cv=5).mean()}')
print(f'GradientBoosting score:{cross_val_score(GDBT, Xtrain, Ytrain, cv=5).mean()}')

# MinMaxScaler
print('\nx do MinMaxScaler')
print(f'LogisticRegression score:{cross_val_score(logistic, Xtrain_mm, Ytrain_mm, cv=5).mean()}')
print(f'DecisionTree score:{cross_val_score(DTree, Xtrain_mm, Ytrain_mm, cv=5).mean()}')
print(f'RandomForest score:{cross_val_score(RForest, Xtrain_mm, Ytrain_mm, cv=5).mean()}')
print(f'GradientBoosting score:{cross_val_score(GDBT, Xtrain_mm, Ytrain_mm, cv=5).mean()}')

# StandarScalar
print('\nx do StandarScalar')
print(f'LogisticRegression score:{cross_val_score(logistic, Xtrain_ss, Ytrain_ss, cv=5).mean()}')
print(f'DecisionTree score:{cross_val_score(DTree, Xtrain_ss, Ytrain_ss, cv=5).mean()}')
print(f'RandomForest score:{cross_val_score(RForest, Xtrain_ss, Ytrain_ss, cv=5).mean()}')
print(f'GradientBoosting score:{cross_val_score(GDBT, Xtrain_ss, Ytrain_ss, cv=5).mean()}')

x do not feature engineering
LogisticRegression score:0.8716666666666667
DecisionTree score:0.8358333333333334
RandomForest score:0.9116666666666667
GradientBoosting score:0.8358333333333334

x do MinMaxScaler
LogisticRegression score:0.9116666666666667
DecisionTree score:0.8366666666666667
RandomForest score:0.9108333333333334
GradientBoosting score:0.8483333333333334

x do StandarScalar
LogisticRegression score:0.8858333333333335
DecisionTree score:0.8483333333333334
RandomForest score:0.8983333333333334
GradientBoosting score:0.8358333333333334


### 提升模型的準確率
***
- L1 or L2正規化(Regularizer)：降低模型多項式Model的複雜程度
  - ：Lasso、Ridge
- 降維：
  - Principal Component Analysis(PCA)：
    1. 當資料內特徵較多時，透過PCA減少特徵數量(降維)，而不會與原始資料特性差異太大。
    2. PCA降維後的特徵並不是簡單的選取原有特徵的一部分，而是將原有的特徵進行縣性組合，得到新的特徵。然後根據特徵值大小判斷對應特徵的重要性，再選取其中重要的一部分，即降維後的特徵。

- Reference:
  1. https://www.zhihu.com/question/38417101

In [0]:
#from sklearn.linear_model import Lasso, Ridge
from sklearn.decomposition import PCA

In [0]:
pca = PCA(n_components=3)
Xtrain_ss_pca = pca.fit_transform(Xtrain_ss)
Xtest_ss_pca = pca.fit_transform(Xtest_ss)

Xtrain_mm_pca = pca.fit_transform(Xtrain_mm)
Xtest_mm_pca = pca.fit_transform(Xtest_mm)

In [23]:
# MinMaxScaler
print('\nx do MinMaxScaler')
print(f'LogisticRegression score:{cross_val_score(logistic, Xtrain_mm_pca, Ytrain_mm, cv=5).mean()}')
print(f'DecisionTree score:{cross_val_score(DTree, Xtrain_mm_pca, Ytrain_mm, cv=5).mean()}')
print(f'RandomForest score:{cross_val_score(RForest, Xtrain_mm_pca, Ytrain_mm, cv=5).mean()}')
print(f'GradientBoosting score:{cross_val_score(GDBT, Xtrain_mm_pca, Ytrain_mm, cv=5).mean()}')

# StandarScalar
print('\nx do StandarScalar')
print(f'LogisticRegression score:{cross_val_score(logistic, Xtrain_ss_pca, Ytrain_ss, cv=5).mean()}')
print(f'DecisionTree score:{cross_val_score(DTree, Xtrain_ss_pca, Ytrain_ss, cv=5).mean()}')
print(f'RandomForest score:{cross_val_score(RForest, Xtrain_ss_pca, Ytrain_ss, cv=5).mean()}')
print(f'GradientBoosting score:{cross_val_score(GDBT, Xtrain_ss_pca, Ytrain_ss, cv=5).mean()}')


x do MinMaxScaler
LogisticRegression score:0.9116666666666667
DecisionTree score:0.8983333333333334
RandomForest score:0.8983333333333334
GradientBoosting score:0.9241666666666667

x do StandarScalar
LogisticRegression score:0.9108333333333334
DecisionTree score:0.8483333333333334
RandomForest score:0.8858333333333335
GradientBoosting score:0.8608333333333335


### 尚未執行PCA的準確率：
***
- x do MinMaxScaler
  1. LogisticRegression score:0.9116666666666667
  2. DecisionTree score:0.8608333333333335
  3. RandomForest score:0.9116666666666667
  4. GradientBoosting score:0.8483333333333334

- x do StandarScalar
  1. LogisticRegression score:0.8858333333333335
  2. DecisionTree score:0.8608333333333335
  3. RandomForest score:0.8983333333333334
  4. GradientBoosting score:0.8358333333333334

### 挑選高準確率模型並進行預測
***
- 挑選最佳模型：
  - X_data做MinMaxScaler和PCA後，使用GradientBoosting來訓練的模型。

- 使用最佳模型進行預測：
  1. 將要預測的資料先進行前處理(補值、標準化、PCA等等)
  2. 將處理好的資料餵到模型進行預測
  3. 將預測結果儲存csv檔

In [0]:
# 定義函數之後有其他新的資料進來可直接呼叫函數
def data_preprocess(train_data):
    # drop a part of columns
    y_train = train_data[['poi']] 
    train_data = train_data.drop(['name', 'email_address', 'poi'], axis=1)
      
    # fill nan
    median_value = dict(train_data.median())
    x_train = train_data.fillna(value=median_value)

    # LabelEncoder
    LabEncoder = LabelEncoder()
    y_train = LabEncoder.fit_transform(y_train)

    # MinMaxScaler
    MMEncoder = MinMaxScaler()
    x_train = MMEncoder.fit_transform(x_train)

    # PCA
    pca = PCA(n_components=3)
    x_train = pca.fit_transform(x_train)

    return x_train, y_train

def predictdata_preprocess(predict_data):
    predict_data = predict_data.drop(['name', 'email_address'], axis=1)
    
    # fill nan
    median_value = dict(predict_data.median())
    predict_data = predict_data.fillna(value=median_value)

    # MinMaxScaler
    MMEncoder = MinMaxScaler()
    predict_data = MMEncoder.fit_transform(predict_data)

    # PCA
    pca = PCA(n_components=3)
    predict_data = pca.fit_transform(predict_data)

    return predict_data

In [25]:
train_data = pd.read_csv(data_root + 'train_data.csv')
x_train, y_train = data_preprocess(train_data)
x_predict = predictdata_preprocess(test_features)
print(f'x_train shape:{x_train.shape}, y_train shape:{y_train.shape}')
print(f'x_predict shape:{x_predict.shape}')

x_train shape:(113, 3), y_train shape:(113,)
x_predict shape:(33, 3)


In [0]:
# 將訓練資料拿來訓練
GDBT.fit(x_train, y_train)
# 模型已訓練完，可進行預測
predict_result = GDBT.predict(x_predict)

In [27]:
# 依照Kaggle格式將結果儲存
column_name = ['name', 'poi']

poi = pd.DataFrame(predict_result)
result = pd.concat([test_features[['name']], poi], axis=1)
result.columns = column_name
result.head()

,name,poi
0,BELDEN TIMOTHY N,1
1,BOWEN JR RAYMOND M,0
2,HANNON KEVIN P,0
3,DELAINEY DAVID W,1
4,CAUSEY RICHARD A,0


In [28]:
# 給路徑與檔名，進行儲存
result.to_csv('content/My Drive/ML100_Colab Notebooks/6.ML100_Midterm test/result.csv')
# 確認路徑下是否儲存csv檔
os.listdir('content/My Drive/ML100_Colab Notebooks/6.ML100_Midterm test/')

['test_features.csv',
 'train_data.csv',
 'Day_51-53_Midterm.ipynb',
 'result.csv']